In [12]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving spark_nlp_for_healthcare.json to spark_nlp_for_healthcare (1).json


In [13]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

In [3]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

! pip install spark-nlp-display

--2021-08-16 07:38:18--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1872 (1.8K) [text/plain]
Saving to: ‘jsl_colab_setup.sh’

jsl_colab_setup.sh  100%[===================>]   1.83K  --.-KB/s    in 0s      

2021-08-16 07:38:19 (29.0 MB/s) - ‘jsl_colab_setup.sh’ saved [1872/1872]

setup Colab for PySpark 3.1.1 and Spark NLP 3.1.3
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/

In [4]:
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import pandas as pd

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.1.1
Spark NLP_JSL Version : 3.1.3


In [10]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [6]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("uncheckedtoken")\

spellModel = ContextSpellCheckerModel\
    .pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("uncheckedtoken")\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

c2doc = Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 

clinical_ner = MedicalNerModel.pretrained("ner_clinical_large", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['PROBLEM'])

sbert_embedder = BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")

icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models")\
    .setInputCols(["ner_chunk", "sbert_embeddings"])\
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")\
    .setReturnCosineDistances(True)

clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
 
sbert_resolver_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        spellModel,
        word_embeddings,
        clinical_ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        icd10_resolver,
        clinical_assertion])

data_ner = spark.createDataFrame([[""]]).toDF("text")

sbert_models = sbert_resolver_pipeline.fit(data_ner)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
spellcheck_clinical download started this may take some time.
Approximate size to download 142.2 MB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
Approximate size to download 1.4 GB
[OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[OK!]


In [5]:
df_light = spark.read.load("data_light.parquet")

In [7]:
from pyspark.sql import functions as F

icd10_df_light = sbert_models.transform(df_light)

In [8]:
df_light_pandas = icd10_df_light.select(F.explode(F.arrays_zip('ner_chunk.result', 
                                                       'ner_chunk.metadata', 
                                                       'icd10cm_code.result', 
                                                       'icd10cm_code.metadata',
                                                       'assertion.result')).alias("cols")) \
                            .select(F.expr("cols['0']").alias("chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['confidence']").alias("confidence"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['4']").alias("assertion")).toPandas()

In [9]:
resolutions = []

for resolution in df_light_pandas['resolutions']:
    resolutions.append(resolution.split(':::'))

df_light_pandas['resolutions'] = resolutions

df_light_pandas['icd_codes_names'] = df_light_pandas['resolutions'].apply(lambda x : x[0])
df_light_pandas.drop("resolutions", axis=1, inplace= True)

In [11]:
df_light_pandas 

,chunk,entity,icd10_code,confidence,assertion,icd_codes_names
0,PharmD,PROBLEM,D099,0.0892,present,ptis category
1,PharmD,PROBLEM,D099,0.0892,present,ptis category
2,Drug-related\tproblems,PROBLEM,T5090,0.3187,present,drug-related disorder
3,numerous\tcomorbidities,PROBLEM,Z789,0.7058,possible,comorbid conditions
4,complications,PROBLEM,T789,0.0981,possible,adverse reactions
5,the\tsymptoms,PROBLEM,R0989,0.1760,hypothetical,cvs symptoms
6,present illness,PROBLEM,R69,0.2752,present,illness
7,hpI,PROBLEM,D588,0.2495,present,hpp
8,present\tillness,PROBLEM,R69,0.2752,present,illness
9,filling,PROBLEM,R938,0.0928,present,filling defect


In [14]:
df = spark.read.load("data.parquet")

In [15]:
from pyspark.sql import functions as F

icd10_df = sbert_models.transform(df)

In [16]:
df_pandas = icd10_df.select(F.explode(F.arrays_zip('ner_chunk.result', 
                                                       'ner_chunk.metadata', 
                                                       'icd10cm_code.result', 
                                                       'icd10cm_code.metadata',
                                                       'assertion.result')).alias("cols")) \
                            .select(F.expr("cols['0']").alias("chunk"),
                                     F.expr("cols['1']['entity']").alias("entity"), 
                                     F.expr("cols['2']").alias("icd10_code"),
                                     F.expr("cols['3']['confidence']").alias("confidence"),
                                     F.expr("cols['3']['all_k_resolutions']").alias("resolutions"),
                                     F.expr("cols['4']").alias("assertion")).toPandas()

In [17]:
resolutions = []

for resolution in df_pandas['resolutions']:
    resolutions.append(resolution.split(':::'))

df_pandas['resolutions'] = resolutions

df_pandas['icd_codes_names'] = df_pandas['resolutions'].apply(lambda x : x[0])
df_pandas.drop("resolutions", axis=1, inplace= True)

In [18]:
df_pandas

,chunk,entity,icd10_code,confidence,assertion,icd_codes_names
0,PharmD,PROBLEM,D099,0.0892,present,ptis category
1,numerous comorbidities,PROBLEM,Z789,0.7058,possible,comorbid conditions
2,complications,PROBLEM,T789,0.0981,possible,adverse reactions
3,present illness,PROBLEM,R69,0.2752,present,illness
4,the symptoms,PROBLEM,R0989,0.1760,hypothetical,cvs symptoms
5,present illness,PROBLEM,R69,0.2752,present,illness
6,present illness,PROBLEM,R69,0.2752,present,illness
7,the patient’s symptoms,PROBLEM,R198,0.1782,present,git symptom changes
8,symptoms,PROBLEM,R0989,0.1270,present,cvs symptoms
9,vomiting,PROBLEM,R111,0.9531,present,vomiting
